In [106]:
from google.colab import drive
drive.mount ('/content/gdrive' )
root_path = '/content/gdrive/My Drive/Retinopathy'
print(root_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Retinopathy


In [107]:
!pip3 install tensorflow_addons
#Hyper parameter optimization
!pip3 install hyperas

In [0]:
import tensorflow as tf
import pathlib
import tensorflow.keras as k
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense,MaxPooling2D, Input, Conv2D, LeakyReLU, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.mobilenet import preprocess_input 
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Model
import cv2
from tensorflow.python.framework.ops import disable_eager_execution
from hyperopt import Trials, STATUS_OK, tpe
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform

/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet
/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet
train images = 413
test images = 103


In [0]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [256, 256])
def process_path(file_path,label):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def image_rotate(file_path,label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    img = tfa.image.rotate(img,90)
    return img, label

def adjust_contrast(file_path,label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    img = tf.image.adjust_contrast(img, 1.5)
    return img, label

def flipped_images(file_path, label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    img = tf.image.random_flip_left_right(img)
    return img, label

In [0]:
def referable_set(filepath,directory,startpoint,endpoint):
  images_list = []
  labels_list = []
  count = 0;
  with open(filepath) as labels:
    for line_no,line in enumerate(labels):
      if line_no > endpoint:
        break
      if (line_no >= startpoint) and (line_no <= endpoint):
        #print(line_no,line)
        x = line.split(',')
        #print(type(x),x)
        if float(x[1]) > 1 :
          labels_list.append(1)
          path = directory + "/"+x[0]+".jpg"
          images_list.append(path)
          count = count + 1
  #print(images_list,"\n")
  #print(labels_list)
  #print(count)
  return images_list,np.array(labels_list, dtype=np.float32)   
          
def non_referable_set(filepath,directory,startpoint,endpoint):
  images_list = []
  labels_list = []
  count = 0;
  with open(filepath) as labels:
    for line_no,line in enumerate(labels):
      if line_no > endpoint:
        break
      if (line_no >= startpoint) and (line_no <= endpoint):
        #print(line_no,line)
        x = line.split(',')
        #print(type(x),x)
        if float(x[1]) <= 1 :
          labels_list.append(0)
          path = directory + "/"+x[0]+".jpg"
          images_list.append(path)
          count = count + 1
  #print(images_list,"\n")
  #print(labels_list)
  #print(count)
  return images_list,np.array(labels_list, dtype=np.float32)  

In [112]:
valid_images_NR,valid_labels_NR = non_referable_set(training_labels,train_directory,1,193)
print(valid_images_NR, valid_labels_NR)
valid_images_R, valid_labels_R = referable_set(training_labels,train_directory, 1,51)
print(valid_images_R, valid_labels_R)
valid_images = valid_images_NR + valid_images_R
valid_labels = np.hstack((valid_labels_NR, valid_labels_R))
print(valid_labels)
print(valid_images)
print("Total valid images:",len(valid_images))

['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_021.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_079.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_105.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_118.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_138.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_139.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_140.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_141.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_142.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_143.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_144.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_145.jpg', '/content/gdrive/My Drive/R

In [0]:
validation_ds = tf.data.Dataset.from_tensor_slices((valid_images,valid_labels))

validation_ds = validation_ds.map(process_path, num_parallel_calls=None)
validation_ds = validation_ds.shuffle(100).batch(10)
#for elem in validation_ds:
 # print(elem)

In [114]:
train_images_NR, train_labels_NR = non_referable_set(training_labels,train_directory,194,413)
print(train_images_NR, train_labels_NR)
train_images_R, train_labels_R = referable_set(training_labels,train_directory, 52,413)
print(train_images_R, train_labels_R)
train_images = train_images_NR + train_images_R
train_labels = np.hstack((train_labels_NR, train_labels_R))
print(train_labels)
print(train_images)
print("Total valid images:",len(train_images))


['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_194.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_195.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_197.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_198.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_199.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_200.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_202.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_203.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_204.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_205.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_206.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_209.jpg', '/content/gdrive/My Drive/R

In [115]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
train_ds = train_ds.shuffle(313)
print(train_ds)
train_ds = train_ds.map(process_path, num_parallel_calls=None)
'''for elem in train_ds:
  print(elem)'''

<ShuffleDataset shapes: ((), ()), types: (tf.string, tf.float32)>


'for elem in train_ds:\n  print(elem)'

In [0]:
train_img_contrast, train_label_contrast = non_referable_set(training_labels,train_directory,1,413)
train_contrast_ds = tf.data.Dataset.from_tensor_slices((train_img_contrast,train_label_contrast))
train_contrast_ds = train_contrast_ds.map(adjust_contrast, num_parallel_calls=None)
combined_training_ds= train_ds.concatenate(train_contrast_ds)
count = 0
#for elem in combined_training_ds:
  #count = count+1;
#print(count)

In [117]:
#flipped images take 104 from each set
flip_images = train_images_NR + train_images_R[:104]
flip_labels = np.hstack((train_labels_NR, train_labels_R[:104]))
print(len(flip_labels))
print("Total images to be flipped:",len(flip_images))
print(flip_images)

208
Total images to be flipped: 208
['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_194.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_195.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_197.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_198.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_199.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_200.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_202.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_203.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_204.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_205.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_206.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_2

In [118]:
train_flipped_ds = tf.data.Dataset.from_tensor_slices((flip_images,flip_labels))
train_flipped_ds = train_flipped_ds.shuffle(208)
train_flipped_ds = train_flipped_ds.map(flipped_images, num_parallel_calls=None)
combined_training_ds1= combined_training_ds.concatenate(train_flipped_ds)
count1 = 0
'''for elem in combined_training_ds1:
  count1 = count1+1
print(count1)'''

'for elem in combined_training_ds1:\n  count1 = count1+1\nprint(count1)'

In [0]:
train_ds = combined_training_ds1.shuffle(675).batch(10)

In [120]:

#test dataset creation
test_images_NR, test_labels_NR = non_referable_set(test_labels,test_directory,1,103)
test_images_R, test_labels_R = referable_set(test_labels,test_directory,1,103)
test_images = test_images_NR + test_images_R
test_labels = np.hstack((test_labels_NR, test_labels_R))
print(test_labels)
print(test_images)
print("Total test images:",len(test_images))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1.]
['/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_029.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_030.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_037.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_038.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_039.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_041.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_043.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet/IDRiD_044.jpg', '/content/gdrive/My Drive/Retinopathy/Origin

In [0]:
test_ds = tf.data.Dataset.from_tensor_slices((test_images,test_labels))
test_ds = test_ds.map(process_path, num_parallel_calls=None)
test_ds = test_ds.shuffle(103).batch(10)

In [0]:
def data():
  root_path = "/content/gdrive/My Drive/Retinopathy"
  train_directory = root_path + "/OriginalImages/TrainingSet"
  test_directory = root_path + "/OriginalImages/TestingSet"

  training_set_path = pathlib.Path(train_directory)
  testing_set_path = pathlib.Path(test_directory)
  training_labels = root_path + "/GroundTruths/IDRiD_Disease Grading_Training Labels.csv"
  test_labels = root_path + "/GroundTruths/IDRiD_Disease Grading_Testing Labels.csv"

  print(training_set_path)
  print(testing_set_path)
  train_image_count = len(list(training_set_path.glob("*.jpg")))
  test_image_count = len(list(testing_set_path.glob("*.jpg")))
  print("train images =",train_image_count)
  print("test images =",test_image_count)
  
  valid_images_NR,valid_labels_NR = non_referable_set(training_labels,train_directory,1,193)
  print(valid_images_NR, valid_labels_NR)
  valid_images_R, valid_labels_R = referable_set(training_labels,train_directory, 1,51)
  print(valid_images_R, valid_labels_R)
  valid_images = valid_images_NR + valid_images_R
  valid_labels = np.hstack((valid_labels_NR, valid_labels_R))
  print(valid_labels)
  print(valid_images)
  print("Total valid images:",len(valid_images))
  validation_ds = tf.data.Dataset.from_tensor_slices((valid_images,valid_labels))

  validation_ds = validation_ds.map(process_path, num_parallel_calls=None)
  validation_ds = validation_ds.shuffle(100).batch(10)
  train_images_NR, train_labels_NR = non_referable_set(training_labels,train_directory,194,413)
  print(train_images_NR, train_labels_NR)
  train_images_R, train_labels_R = referable_set(training_labels,train_directory, 52,413)
  print(train_images_R, train_labels_R)
  train_images = train_images_NR + train_images_R
  train_labels = np.hstack((train_labels_NR, train_labels_R))
  print(train_labels)
  print(train_images)
  print("Total valid images:",len(train_images))
  train_ds = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
  train_ds = train_ds.shuffle(313)
  print(train_ds)
  train_ds = train_ds.map(process_path, num_parallel_calls=None)
  train_img_contrast, train_label_contrast = non_referable_set(training_labels,train_directory,1,413)
  train_contrast_ds = tf.data.Dataset.from_tensor_slices((train_img_contrast,train_label_contrast))
  train_contrast_ds = train_contrast_ds.map(adjust_contrast, num_parallel_calls=None)
  combined_training_ds= train_ds.concatenate(train_contrast_ds)

  flip_images = train_images_NR + train_images_R[:104]
  flip_labels = np.hstack((train_labels_NR, train_labels_R[:104]))
  print(len(flip_labels))
  print("Total images to be flipped:",len(flip_images))
  print(flip_images)
  train_flipped_ds = tf.data.Dataset.from_tensor_slices((flip_images,flip_labels))
  train_flipped_ds = train_flipped_ds.shuffle(208)
  train_flipped_ds = train_flipped_ds.map(flipped_images, num_parallel_calls=None)
  combined_training_ds1= combined_training_ds.concatenate(train_flipped_ds)
  train_ds = combined_training_ds1.shuffle(675).batch(10)
  return train_ds,validation_ds

In [0]:
def create_model(training_ds, validating_ds):
  learning_rate = 0.001
  inputs = k.layers.Input(shape=(256,256,3))
  conv_layer1 = Conv2D(32,5,(2,2),activation='relu')(inputs)
  FMP_layer1 = tf.nn.fractional_max_pool(conv_layer1,1.5,True)[0]
  conv_layer2 = Conv2D(64,3,(1,1),activation='relu')(FMP_layer1)
  FMP_layer2 = tf.nn.fractional_max_pool(conv_layer2,1.8, True)[0]
  conv_layer3 = Conv2D(128,3,(1,1),activation='relu')(FMP_layer2)
  FMP_layer3 = tf.nn.fractional_max_pool(conv_layer3,1.5)[0]
  conv_layer4 = Conv2D(160,3,(1,1),activation='relu')(FMP_layer3)
  
  GAP = k.layers.GlobalAveragePooling2D()(conv_layer4)
  flatten = k.layers.Flatten()(GAP)

  dense_layer1 = Dense(64,activation='relu')(flatten)
  dropout_layer1 = Dropout({{uniform(0, 1)}})(dense_layer1)

  dense_layer2 = Dense(32,activation='relu')(dropout_layer1)
  output = k.layers.Dense(1, activation='sigmoid')(dense_layer2)

  model = k.models.Model(inputs=inputs, outputs=output)
  model.compile(optimizer= {{choice(['rmsprop', 'adam', 'sgd'])}} ,loss="binary_crossentropy", metrics= ['accuracy'])
  model.summary()
  result = model.fit(train_ds,validation_data = validation_ds,epochs = 150,verbose = 2)
  validation_acc = np.amax(result.history['val_acc']) 
  print('Best validation acc of epoch:', validation_acc)
  return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


In [3]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          functions = [non_referable_set,referable_set,process_path,adjust_contrast,flipped_images,decode_img],
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='gdrive/My Drive/Retinopathy/Test_Model')
training_ds,valid_ds = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(valid_ds))
print("Best performing model chosen hyper-parameters:")
print(best_run)

NameError: ignored